# Anàlisi de rendiment de sucursals i productes

In [ ]:
# Ets analista de dades d’una empresa amb diverses sucursals que venen productes de diferents categories.
# El teu objectiu és analitzar el rendiment de vendes i beneficis al llarg dels anys i per sucursal, utilitzant pandas.

In [ ]:
# Importa pandas com a pd i numpy com a np. 

In [ ]:
# Creació del DataFrame

# Crea un nou DataFrame anomenat sales_df amb les columnes següents:

# branch (sucursal) → "Barcelona", "Madrid", "València", "Sevilla"

# category → "Tecnologia", "Moda", "Alimentació"

# year → un valor entre 2018 i 2023

# units_sold → nombre d’unitats venudes (enter)

# price_per_unit → preu per unitat (float)

# cost_per_unit → cost de producció o compra (float)

# Genera les dades aleatòriament amb numpy per a unes 100 files.
# Afegeix una columna calculada anomenada profit = (price_per_unit - cost_per_unit) * units_sold.

# Mostra les 5 primeres files.



,branch,category,year,units_sold,price_per_unit,cost_per_unit,profit
0,València,Moda,2021,581,470.526999,198.171502,158238.543660
1,Barcelona,Moda,2022,890,477.194646,190.968293,254741.453829
2,Barcelona,Alimentació,2020,11,457.857873,71.761142,4247.064039
3,Barcelona,Moda,2022,906,188.228557,175.239105,11768.443402
4,València,Alimentació,2022,313,12.651025,161.206380,-46497.825919


In [ ]:
# Estadístiques generals

# Fes servir .describe() per obtenir una visió general de les columnes numèriques.

,year,units_sold,price_per_unit,cost_per_unit,profit
count,100.000000,100.000000,100.000000,100.000000,100.000000
mean,2020.690000,483.740000,261.801711,201.739792,23084.244762
std,1.624901,298.507825,147.761030,118.867148,104275.426705
min,2018.000000,11.000000,12.124777,7.302548,-255281.067345
25%,2019.000000,213.750000,146.020260,106.354529,-22386.813370
50%,2021.000000,474.000000,273.111740,203.165143,14364.446900
75%,2022.000000,736.500000,389.264398,303.245071,60990.496195
max,2023.000000,977.000000,495.076656,396.230541,358055.292909


In [ ]:
# Agrupació per categoria i any

# Agrupa per category i year i calcula:

# Total de vendes (total_sales = sumatori de price_per_unit * units_sold)

# Benefici mitjà (profit.mean())

# Unitats venudes totals (units_sold.sum())

# Desa-ho en un DataFrame anomenat summary_df.

,category,year,total_sales,profit_mean,units_sold_sum
0,Alimentació,2018,1.841104e+05,2989.453726,954
1,Alimentació,2019,4.368045e+05,-65454.635808,3983
2,Alimentació,2020,4.256393e+05,7817.140117,1725
3,Alimentació,2021,1.077750e+06,20849.326986,4297
4,Alimentació,2022,4.892213e+05,8417.265037,2045


In [ ]:
# Afegir columna calculada i ordenar

# A summary_df, crea una columna margin_percent = (profit_mean / (total_sales / units_sold_sum)) * 100.
# Ordena el resultat pel marge més alt (margin_percent) descendent.

,category,year,total_sales,profit_mean,units_sold_sum,margin_percent
8,Moda,2020,2.233743e+05,213678.900919,462,44194.724457
12,Tecnologia,2018,7.585407e+05,138344.451703,2292,41802.039030
7,Moda,2019,9.417157e+05,67515.818700,3037,21773.612945
13,Tecnologia,2019,5.699962e+05,35048.836490,3388,20832.674265
10,Moda,2022,1.630269e+06,60166.337012,5011,18493.483736


In [ ]:
# Agrupació per sucursal i categoria

# Agrupa per branch i category, i calcula:

# total_sales

# profit_total

# avg_price

# avg_cost

# Desa-ho en branch_summary.

,branch,category,total_sales,profit_total,avg_price,avg_cost
0,Barcelona,Alimentació,8.922963e+05,-69460.696890,277.285641,207.627552
1,Barcelona,Moda,1.470525e+06,451307.014464,250.419184,177.115446
2,Barcelona,Tecnologia,6.503912e+05,-233960.509100,204.965885,220.194491
3,Madrid,Alimentació,4.608572e+05,172012.670278,245.895186,110.112879
4,Madrid,Moda,1.062760e+06,228748.742140,387.727043,271.726745


In [ ]:
# Filtratge de resultats

# Mostra només les sucursals on el profit_total superi la mitjana de tots els beneficis totals.

,branch,category,total_sales,profit_total,avg_price,avg_cost
1,Barcelona,Moda,1.470525e+06,4.513070e+05,250.419184,177.115446
4,Madrid,Moda,1.062760e+06,2.287487e+05,387.727043,271.726745
5,Madrid,Tecnologia,1.131863e+06,2.403541e+05,264.912807,216.876860
7,Sevilla,Moda,5.470969e+05,3.380417e+05,308.771852,203.980928
8,Sevilla,Tecnologia,1.419585e+06,1.096992e+06,304.216924,111.683386
10,València,Moda,1.491313e+06,4.991435e+05,323.226152,243.941959


In [ ]:
# Transformació de dades

# Afegeix una nova columna a sales_df que representi el benefici normalitzat per categoria, és a dir, cada valor de profit dividit pel màxim profit dins la seva categoria.

,category,profit,normalized_profit
0,Moda,158238.543660,0.621173
1,Moda,254741.453829,1.000000
2,Alimentació,4247.064039,0.021965
3,Moda,11768.443402,0.046198
4,Alimentació,-46497.825919,-0.240473


In [ ]:
# Filtrat amb funció personalitzada

# Defineix una funció filter_func(x) que retorni només els grups (per categoria) on el desviament estàndard del benefici (profit.std()) sigui superior a 5000.
# Aplica-la amb .groupby('category').filter(filter_func).

array(['Moda', 'Alimentació', 'Tecnologia'], dtype=object)

In [ ]:
# Anàlisi addicional per dècada

# Crea una nova columna decade agrupant els anys en dècades (2010s, 2020s, etc.).
# Agrupa per decade i category i mostra el nombre total d’unitats venudes i el benefici mitjà.

,decade,category,total_units,avg_profit
0,2010s,Alimentació,4937,-51765.817901
1,2010s,Moda,5522,50931.661381
2,2010s,Tecnologia,5680,73784.692195
3,2020s,Alimentació,10317,20982.205218
4,2020s,Moda,9902,47687.423742
5,2020s,Tecnologia,12016,6921.071773
